In [1]:
#%pip install scikit-allel
import pandas as pd 
import numpy as np
import allel
import tabix
import sys
from collections import defaultdict

In [2]:
chromosome_number = sys.argv[1]
yri="NA18486"
gbr="HG00148"

yri_index = 1764
gbr_index = 54

chr_size = pd.read_csv('/home/jbazanwilliamson/Enhancer-Regions-Dplus/size_chr', sep='\t', dtype=str, names=['chr','start','end'])
data_human = tabix.open('/home/jbazanwilliamson/thousandgenomes/chr1.biallelic.vcf.gz')
data_archaic = tabix.open('/courses/ehuerta-sanchez/AltaiNea_Den_combined/AltaiNea_Den_combined.1.vcf.gz')


In [3]:
import os
os.path.exists('/courses/ehuerta-sanchez/AltaiNea_Den_combined/AltaiNea_Den_combined.1.vcf.gz')

True

In [4]:
#Get African and European alleles
data_human = data_human.query("1",0,249250621)
new_data = defaultdict(defaultdict)

for i in data_human:
    if("0|0" in i[54]+i[1764] and "1|1" in i[54]+i[1764]):
        new_data[i[1]].update({"human reference":i[3],"human alt":i[4],"yri":i[1764],
        "gbr":i[54]})

KeyboardInterrupt: 

In [ ]:
#Lesly's function to get chimp allele
def get_chimp_allele(string):
  index=string.find("CAnc")
  if index < 0:
    return("")
  substring=string[index:]
  chimp_allele=substring.split(";")[0].split("=")[1]
  return(chimp_allele)
#
data_new_archaic = data_archaic.query("1",0,249250621) 


#Get neanderthal and chimp
for spline in data_new_archaic:
    if "." in spline[9].split(":")[0]:
      continue
    #is it from biallelic human SNPs
    if(spline[1] in list(new_data.keys())):
            new_data[spline[1]].update({"archaic reference":spline[3],"archaic alt":spline[4], "chimp allele":get_chimp_allele(spline[7]),"neanderthal":spline[9].split(":")[0]})

In [197]:
#Get alleles for the 4 populations
df = (pd.DataFrame.from_dict(new_data)).T
df['chimp allele'].replace('', np.nan, inplace=True)
df['chimp allele'].replace('-', np.nan, inplace=True)
df.dropna(subset=['chimp allele'], inplace=True)
#Remove heterozygote from neanderthal allele
df['neanderthal'].replace('0/1', np.nan, inplace=True)
df['neanderthal'].replace('1/0', np.nan, inplace=True)
df.dropna(subset=['neanderthal'], inplace=True)  

#Change genotype to allele 
df.loc[df['yri'] == "0|0", 'yri'] = df["human reference"]
df.loc[df['yri'] == "1|1", 'yri'] = df["human alt"]

df.loc[df['gbr'] == "0|0", 'gbr'] = df["human reference"]
df.loc[df['gbr'] == "1|1", 'gbr'] = df["human alt"]

df.loc[df['neanderthal'] == "0/0", 'neanderthal'] = df["archaic reference"]
df.loc[df['neanderthal'] == "1/1", 'neanderthal'] = df["archaic alt"]


df.drop(columns=['human reference', 'human alt', 'archaic reference', 'archaic alt'], inplace=True)
df
               

,yri,gbr,chimp allele,neanderthal
748061,A,T,T,T
758274,C,G,G,G
758770,A,C,C,C
772028,A,G,G,G
776112,C,T,T,T
...,...,...,...,...
2215121,T,C,C,C
2215211,C,T,C,C
2215909,C,A,A,A
2220425,T,C,C,C


In [198]:
#Get positions that have 2 differents alleles 
for i, row in df.iterrows():
    if len(row.T[row.T.notnull()].unique())!=2:
        df.drop([i], inplace=True)    

In [199]:
#Change alleles to A,B frequency

df.loc[df['yri'] == df["chimp allele"], 'yri'] = "A"
df.loc[df['yri'] != df["chimp allele"], 'yri'] = "B"

df.loc[df['gbr'] == df["chimp allele"], 'gbr'] = "A"
df.loc[df['gbr'] != df["chimp allele"], 'gbr'] = "B"

df.loc[df['neanderthal'] == df["chimp allele"], 'neanderthal'] = "A"
df.loc[df['neanderthal'] != df["chimp allele"], 'neanderthal'] = "B"

df["chimp allele"]="A"

df

,yri,gbr,chimp allele,neanderthal
748061,B,B,A,B
758274,B,B,A,B
758770,B,B,A,B
772028,B,B,A,B
776112,B,B,A,B
...,...,...,...,...
2215121,B,B,A,B
2215211,B,B,A,B
2215909,B,A,A,A
2220425,B,B,A,B


In [201]:
#intermediate file as csv

df.to_csv('/home/jbazanwilliamson/Enhancer-Regions-Dplus/chr_1_intermediate.csv')